In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, LSTM, Dense,Bidirectional
from tensorflow.keras.layers import Conv1D, Reshape, Bidirectional, LSTM, Dropout, Dense, LayerNormalization
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
import tensorflow as tf
import xgboost as xgb
from tensorflow.keras.layers import Conv1D, Reshape, Bidirectional, GRU, Dropout, Dense, LayerNormalization




In [ ]:
corpus = pd.read_csv("model_data.csv")

In [ ]:
features = ['company_code',
 'dale_chill_formula',
 'flesch_score',
 'gunnning_fog_score',
 'verb_count',
 'noun_count',
 'adjective_count',
 'len_text',
 'len_text_words',
 'len_mentions',
 'score_group',
 'prev_likes',
 'avg_comp_likes',
 'moving_avg_likes',
 'Followers',
 'hashtag_count',
 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
X = corpus[features]
y = corpus["like_count"]

In [ ]:
kl = tf.keras.losses.KLDivergence()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('vinai/bertweet-base')

In [ ]:
X_text = tokenizer(corpus['lemmatized_text'].tolist(),add_special_tokens=True, truncation=True, padding='max_length', max_length=512)


In [ ]:
X_combined = pd.concat([X, pd.DataFrame(X_text["input_ids"])], axis =1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_combined.values, y.values, test_size=0.2, random_state=1)

In [ ]:
X_train = np.reshape(X_train, (-1, X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (-1, X_test.shape[0], X_test.shape[1]))

X_train = np.transpose(X_train, (1, 0, 2))
X_test = np.transpose(X_test, (1, 0, 2))

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
# Create the model
model_lstm = Sequential()

# 1st Conv1D layer with 128 filters, kernel size of 3, ReLU activation, and 'same' padding
model_lstm.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(1, 534), padding="same"))

# Add LayerNormalization to stabilize training and improve generalization
model_lstm.add(LayerNormalization())

# 2nd Conv1D layer with 256 filters, kernel size of 3, ReLU activation, and 'same' padding
model_lstm.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))

# Add LayerNormalization again
model_lstm.add(LayerNormalization())

# Reshape the output to fit the LSTM layer input requirements
model_lstm.add(Reshape((-1, 256)))

# Add a Bidirectional LSTM layer with 128 units and return sequences
model_lstm.add(Bidirectional(LSTM(units=128, return_sequences=True)))

# Add a dropout layer with a rate of 0.3
model_lstm.add(Dropout(0.3))

# Add a second Bidirectional LSTM layer with 64 units
model_lstm.add(Bidirectional(LSTM(units=64)))

# Add a dropout layer with a rate of 0.3
model_lstm.add(Dropout(0.3))

# Add a Dense layer with 32 units and ReLU activation
model_lstm.add(Dense(units=32, activation='relu'))

# Add a dropout layer with a rate of 0.3
model_lstm.add(Dropout(0.3))

# And a Dense output layer with 1 unit and linear activation for regression
model_lstm.add(Dense(units=1, activation='linear'))

In [ ]:
# Create the model
model_gru = Sequential()

# 1st Conv1D layer with 128 filters, kernel size of 3, ReLU activation, and 'same' padding
model_gru.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(1, 512), padding="same"))

# Add LayerNormalization to stabilize training and improve generalization
model_gru.add(LayerNormalization())

# 2nd Conv1D layer with 256 filters, kernel size of 3, ReLU activation, and 'same' padding
model_gru.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))

# Add LayerNormalization again
model_gru.add(LayerNormalization())

# Reshape the output to fit the GRU layer input requirements
model_gru.add(Reshape((-1, 256)))

# Add a Bidirectional GRU layer with 128 units and return sequences
model_gru.add(Bidirectional(GRU(units=128, return_sequences=True)))

# Add a dropout layer with a rate of 0.3
model_gru.add(Dropout(0.3))

# Add a second Bidirectional GRU layer with 64 units
model_gru.add(Bidirectional(GRU(units=64)))

# Add a dropout layer with a rate of 0.3
model_gru.add(Dropout(0.3))

# Add a Dense layer with 32 units and ReLU activation
model_gru.add(Dense(units=32, activation='relu'))

# Add a dropout layer with a rate of 0.3
model_gru.add(Dropout(0.3))

# Add a Dense output layer with 1 unit and linear activation for regression
model_gru.add(Dense(units=1, activation='linear'))

# Compile the model with the Adam optimizer, mean squared error loss, and mean absolute error metric
model_gru.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
model_lstm.compile(
    optimizer='adam',  # Optimizer
    # Loss function to minimize
    loss='mse',
    # List of metrics to monitor
    metrics=[kl,tf.keras.metrics.MeanAbsoluteError() ]
)

In [ ]:
model_gru.compile(
    optimizer='adam',  # Optimizer
    # Loss function to minimize
    loss='mse',
    # List of metrics to monitor
    metrics=[kl,tf.keras.metrics.MeanAbsoluteError() ]
)

In [ ]:
print("Fit model on training data")
history = model_lstm.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=3,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    #validation_data=(X_val, y_val),
)

In [ ]:
print("Fit model on training data")
history = model_gru.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=3,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    #validation_data=(X_val, y_val),
)

In [ ]:
lstm_predictions = model_lstm.predict(X_test)

In [ ]:
gru_predictions = model_gru.predict(X_test)

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)


In [ ]:
xgb_model.fit(X_train, y_train) # or gru_predictions

In [ ]:
xgb_preds = xgb_model.predict(X_test)

In [ ]:
ensemble_preds = (lstm_predictions + xgb_preds) / 2
ensemble_preds_xgb_lstm = ensemble_preds[0]

In [ ]:
ensemble_preds = (gru_predictions + xgb_preds) / 2
ensemble_preds_xgb_gru = ensemble_preds[0]

In [ ]:
r2_en_lstm_xgboost = r2_score(y_test, ensemble_preds_xgb_lstm)
r2_en_gru_xgboost = r2_score(y_test, ensemble_preds_xgb_gru)

In [ ]:
print(r2_en_lstm_xgboost,r2_en_gru_xgboost)